In [1]:
################################################# import libraries ###########################################
%load_ext jupyternotify

import pandas as pd
import os
from nltk.corpus import stopwords
import string
import re
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import adjusted_rand_score
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import plotly.plotly as py
import operator
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud
import time
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.cluster import AgglomerativeClustering
from gensim.models import KeyedVectors
from gensim.test.utils import datapath
from sklearn.cluster import MiniBatchKMeans
import pickle
from gensim.models import Word2Vec


<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import codecs
from tqdm import tqdm

def load_dataset(path):
    dataset = pd.DataFrame(columns=['X', 'y1', 'y2'])
    #print('Loading dataset...')
    with codecs.open(path, "r", encoding='utf-8', errors='ignore') as fdata:
        for line in tqdm(fdata.readlines()):
            line_split = line.split()
            formated = ' '.join(line_split[:-2])
            dataset.loc[-1] = [formated, line_split[-2], line_split[-1]]  # adding a row
            dataset.index = dataset.index + 1  # shifting index
            dataset = dataset.sort_index()  # sorting by index
    return dataset

In [3]:
from nltk.tokenize import TweetTokenizer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

def tokenize(text):
    tkz = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True)
    stemmer = SnowballStemmer("german")
    output = []
    tokens = tkz.tokenize(text)
    for token in tokens:
        if token not in stopwords_german:
            if len(token) > 1:
                if token[0] == '#':
                    token = token[1:]
                output.append(token)
    return output

In [4]:
dataset = load_dataset(path = '/home/text_mining_project/text_mining_project_2018/evaluation/germeval2018.training.txt')

100%|██████████| 5009/5009 [00:14<00:00, 346.13it/s]


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


X = dataset['X'].values
y = dataset['y1'].values

lb = LabelEncoder()
lb.fit(['OFFENSE','OTHER'])
y = np.ones(len(y)) - lb.transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [6]:
stopwords_german = set(stopwords.words('german'))

def preprocess(X):
    return [tokenize(sentence) for sentence in X]

X_train = preprocess(X_train)
X_test = preprocess(X_test)

In [7]:
def load_w2vmodel(path):
    with open(path, "rb") as lines:
        w2v = {line.split()[0]: np.array(list(map(float, line.split()[1:])))
                for line in tqdm(lines)}
    return w2v

In [8]:
# model = KeyedVectors.load_word2vec_format(datapath("/home/text_mining_project/twitter-de_d100_w5_min10.bin"), binary=True)
model = Word2Vec.load("word2vec_nostem.model")


In [9]:
model.wv.most_similar('untermensch')

[('fleig', 0.47227969765663147),
 ('sonderbedeutung', 0.4633481502532959),
 ('vorwirft', 0.4559037387371063),
 ('red-pack', 0.4509229063987732),
 ('sparxsystems', 0.4493067264556885),
 ('sachggegenstand', 0.44587182998657227),
 ('multiversum', 0.44581305980682373),
 ('dazugehöre', 0.44405651092529297),
 ('blaumacher', 0.44281986355781555),
 ('gegenüb', 0.4426388144493103)]

In [10]:
model.wv.similarity('afd', 'gauland')

0.5590099346449514

In [11]:
kmeans_args = {
    'n_clusters': 1000,
}

# clustering = KMeans(**kmeans_args).fit_predict(model.wv.vectors)

clustering = MiniBatchKMeans(**kmeans_args).fit_predict(model.wv.vectors)
pickle.dump(clustering, open('./minibatchkmeans.bin', 'wb'))

/home/koogleschreiber/.local/lib/python3.5/site-packages/sklearn/cluster/k_means_.py:1418: RuntimeWarning:

init_size=300 should be larger than k=1000. Setting it to 3*k



In [12]:
%%notify -o
msg = 'finished clustering'
msg

'finished clustering'

<IPython.core.display.Javascript object>

In [13]:
word2centroid = {k: v for k, v in zip(model.wv.index2word, clustering)}

In [14]:
from numpy import zeros

def make_bag_of_centroids(X, word_centroid_map, cluster_size):
    centroids_list = []
    for sentence in X:
        centroids = zeros(cluster_size, dtype="float32")
        for word in sentence:
            if word in word_centroid_map:
                centroids[word_centroid_map[word]] += 1
        centroids_list.append(centroids)
    return centroids_list

as_centroid = lambda s: make_bag_of_centroids(s, word2centroid, kmeans_args['n_clusters'])

In [15]:
from xgboost import XGBClassifier
from sklearn.preprocessing import scale
import pandas as pd

fit = XGBClassifier().fit(scale(as_centroid(X_train)), y_train)

In [16]:
y_predict = fit.predict(scale(as_centroid(X_test)))

/home/koogleschreiber/.local/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning:

The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.



In [17]:
from sklearn.metrics import f1_score
f1=f1_score(y_predict, y_test)
print("F1-Score predictions XGBClassifier: ", f1)

F1-Score predictions XGBClassifier:  0.2616822429906542


In [18]:
import warnings

seed = 42
k = 7
jobs = -1

warnings.filterwarnings("ignore")

In [19]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.multiclass import OneVsRestClassifier

from sklearn.model_selection import cross_val_score


names = ["Linear SVM", "SGDClassifier", "BernoulliNB", "LogisticRegression",
         "KNeighborsClassifier", "AdaBoostClassifier", "Random Forest", "Decision Tree"]

classifiers = [
    LinearSVC(random_state=seed),
    SGDClassifier(max_iter=1000, tol=None),
    BernoulliNB(),
    LogisticRegression(random_state=seed, solver='sag', max_iter=1000),
    KNeighborsClassifier(),
    AdaBoostClassifier(),
    RandomForestClassifier(random_state=seed),
    DecisionTreeClassifier(random_state=seed)
]

print('Searching best estimator...')
print()
best_classifier = None
for name, clf in zip(names, classifiers):
    scores = cross_val_score(clf, scale(as_centroid(X_train)), y_train, cv=k, n_jobs=jobs)
    print('Mean accuracy %s: %0.3f (+/- %0.3f)' % (name, scores.mean(), scores.std() * 2))
    if not best_classifier:
        best_classifier = (name, scores.mean())
    else:
        if best_classifier[1] < scores.mean():
            best_classifier = (name, scores.mean())
print()
print('Best estimator: %s (mean acc %0.3f, %d-fold cross-validation)' % (best_classifier[0], best_classifier[1], k))

Searching best estimator...

Mean accuracy Linear SVM: 0.673 (+/- 0.044)
Mean accuracy SGDClassifier: 0.664 (+/- 0.037)
Mean accuracy BernoulliNB: 0.701 (+/- 0.035)
Mean accuracy LogisticRegression: 0.671 (+/- 0.039)
Mean accuracy KNeighborsClassifier: 0.656 (+/- 0.020)
Mean accuracy AdaBoostClassifier: 0.683 (+/- 0.032)
Mean accuracy Random Forest: 0.666 (+/- 0.029)
Mean accuracy Decision Tree: 0.636 (+/- 0.018)

Best estimator: BernoulliNB (mean acc 0.701, 7-fold cross-validation)


In [20]:
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer

print('Searching best estimator (F1 score) ...')
print()
best_classifier = None
for name, clf in zip(names, classifiers):
    scores = cross_val_score(clf, scale(as_centroid(X_train)), y_train, cv=k, n_jobs=jobs, scoring='f1')
    print('Mean F1 score %s: %0.3f (+/- %0.3f)' % (name, scores.mean(), scores.std() * 2))
    if not best_classifier:
        best_classifier = (name, scores.mean())
    else:
        if best_classifier[1] < scores.mean():
            best_classifier = (name, scores.mean())
print()
print('Best estimator: %s (mean F1 score %0.3f, %d-fold cross-validation)' % (best_classifier[0], best_classifier[1], k))

Searching best estimator (F1 score) ...

Mean F1 score Linear SVM: 0.481 (+/- 0.066)
Mean F1 score SGDClassifier: 0.464 (+/- 0.069)
Mean F1 score BernoulliNB: 0.490 (+/- 0.067)
Mean F1 score LogisticRegression: 0.476 (+/- 0.057)
Mean F1 score KNeighborsClassifier: 0.294 (+/- 0.070)
Mean F1 score AdaBoostClassifier: 0.391 (+/- 0.086)
Mean F1 score Random Forest: 0.340 (+/- 0.070)
Mean F1 score Decision Tree: 0.451 (+/- 0.035)

Best estimator: BernoulliNB (mean F1 score 0.490, 7-fold cross-validation)
